### Import des Bibliothéques
---

In [ ]:
import pandas as pd
import numpy as np
import pickle
import os
import boto3
import pickle # moyen le plus simple et direct pour sauvegarder et recharger un modèle Python entier.
import statsmodels.api as sm

from dotenv import load_dotenv
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error, mean_absolute_percentage_error



### Chargement fichier df_reduced.csv du S3 
---

In [80]:
load_dotenv("secrets.env")

AWS_KEY = os.getenv("AWS_ACCESS_KEY_ID")
AWS_SECRET = os.getenv("AWS_SECRET_ACCESS_KEY")
BUCKET_NAME = os.getenv("AWS_BUCKET_NAME")
NEON_CONN = os.getenv("NEON_CONN")

s3 = boto3.client(
    "s3",
    aws_access_key_id=AWS_KEY,
    aws_secret_access_key=AWS_SECRET
)

# chargement fichier du S3
s3.download_file(BUCKET_NAME, "df_reduced.csv", "df_reducedS3.csv")

# Lecture des fichiers 
df_reducedS3 = pd.read_csv("df_reducedS3.csv")

display(df_reducedS3.head())



,Département,année,densité,pop_globale,tranche_age_0-24,tranche_age_25-59,tranche_age_60+,csp1_agriculteurs,csp2_artisans_commerçant_chef_entreprises,csp3_cadres_professions_intellectuelles,...,nb_salaries_secteur_industrie,nb_salaries_secteur_service,nbre_entreprises_agricole,nbre_entreprises_industrie,nbre_entreprises_service,Total_autres_dechets,Déblais_gravats,Déchets_verts,Encombrants,Matériaux_recyclables
0,Ain,2009,101.8,588857,188068,281744,119045,4017,17226,37333,...,68343,141408,3092,6263,15605,4803,37593,45013,34729,25405
1,Aisne,2009,73.4,539547,170369,249910,119268,5250,11668,19234,...,44128,129961,3720,3913,11215,1180,24380,24539,31593,10542
2,Allier,2009,46.6,342559,86913,152570,103076,6019,9743,12045,...,31537,92422,3325,3011,8474,1867,14374,18612,11932,8564
3,Alpes-de-Haute-Provence,2009,23.0,159510,43500,70222,45788,2136,6704,7210,...,10803,44575,1473,2096,6415,803,11228,8280,9006,7327
4,Hautes-Alpes,2009,24.5,135883,37235,62716,35932,2214,5658,6070,...,8607,46547,1333,1869,5980,1048,8509,6121,7938,10344


### Modéle OLS
---

In [81]:

# --- Targets à prédire ---
targets = [
    'Total_autres_dechets',
    'Déblais_gravats',
    'Déchets_verts',
    'Encombrants',
    'Matériaux_recyclables'
]

# --- DataFrame pour stocker les résultats ---
df_results = pd.DataFrame(columns=['cible','R2_train','R2_test','MAE','RMSE'])

# --- Boucle sur chaque target ---
for target in targets:
    print(f"--- Traitement de {target} ---")
    
    X = df_reducedS3.drop(columns=[target])
    y = df_reducedS3[target]
    
    # Diviser train/test
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Identifier colonnes numériques et catégorielles
    numerical_columns = X.select_dtypes(include=['float64', 'int64']).columns.tolist()
    categorical_columns = X.select_dtypes(include=['object', 'category', 'bool']).columns.tolist()

    # --- ColumnTransformer pour standardiser et encoder ---
    preprocessor = ColumnTransformer(transformers=[
        ('num', StandardScaler(), numerical_columns),
        ('cat', OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore'), categorical_columns)
    ])
    
    # Encoder train et test (converti en dense)
    X_train_enc = preprocessor.fit_transform(X_train)
    X_test_enc = preprocessor.transform(X_test)
    
    # Ajouter constante pour OLS
    X_train_enc_const = sm.add_constant(X_train_enc)
    X_test_enc_const = sm.add_constant(X_test_enc, has_constant='add')
    
    # --- Modèle OLS ---
    ols_model = sm.OLS(y_train, X_train_enc_const).fit()
    
    # Prédictions
    y_train_pred = ols_model.predict(X_train_enc_const)
    y_test_pred = ols_model.predict(X_test_enc_const)
    
    # Calcul des métriques
    r2_train = r2_score(y_train, y_train_pred)
    r2_test = r2_score(y_test, y_test_pred)
    mae = mean_absolute_error(y_test, y_test_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
    mape = round(mean_absolute_percentage_error(y_test, y_test_pred),4) * 100  # en %
    
    # Stocker les résultats
    df_results = pd.concat([df_results, pd.DataFrame([{
        'cible': target,
        'R2_train': r2_train,
        'R2_test': r2_test,
        'MAE': mae,
        'RMSE': rmse,
        'MAPE': mape 
    }])], ignore_index=True)
    
    # Sauvegarder le modèle
    with open(f'model_ols_{target}.pkl', 'wb') as f:
        pickle.dump(ols_model, f)

df_results.head()

--- Traitement de Total_autres_dechets ---
--- Traitement de Déblais_gravats ---
--- Traitement de Déchets_verts ---


C:\Users\natha\AppData\Local\Temp\ipykernel_7620\333000565.py:56: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_results = pd.concat([df_results, pd.DataFrame([{


--- Traitement de Encombrants ---
--- Traitement de Matériaux_recyclables ---


,cible,R2_train,R2_test,MAE,RMSE,MAPE
0,Total_autres_dechets,0.852505,0.763521,2945.929513,4790.515765,49.37
1,Déblais_gravats,0.973181,0.974606,4248.225994,6408.328741,19.43
2,Déchets_verts,0.987394,0.933627,4522.437305,9102.058983,14.84
3,Encombrants,0.970889,0.901533,5023.831558,8634.605841,13.04
4,Matériaux_recyclables,0.982392,0.963132,2166.528217,2995.279314,13.58


In [82]:
ols_model.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                              OLS Regression Results                             
=================================================================================
Dep. Variable:     Matériaux_recyclables   R-squared:                       0.982
Model:                               OLS   Adj. R-squared:                  0.977
Method:                    Least Squares   F-statistic:                     199.8
Date:                   Thu, 09 Oct 2025   Prob (F-statistic):          1.64e-309
Time:                           16:44:35   Log-Likelihood:                -4861.3
No. Observations:                    537   AIC:                             9959.
Df Residuals:                        419   BIC:                         1.046e+04
Df Model:                            117                                         
Covariance Type:               nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       3.225e+04   4726.586      6.823      0.000     2.3e+04    4.15e+04
x1          -177.8087    249.139     -0.714      0.476    -667.526     311.909
x2          1.072e+04   1.17e+04      0.919      0.359   -1.22e+04    3.36e+04
x3         -3539.8696   1616.995     -2.189      0.029   -6718.302    -361.437
x4          4596.5505   1.26e+04      0.365      0.715   -2.02e+04    2.94e+04
x5         -1.303e+04   1.05e+04     -1.236      0.217   -3.38e+04    7696.737
x6          5585.9104   6619.354      0.844      0.399   -7425.368    1.86e+04
x7         -1952.7729   1980.619     -0.986      0.325   -5845.960    1940.415
x8         -3228.4779   3158.757     -1.022      0.307   -9437.464    2980.508
x9          1.374e+04   5721.074      2.402      0.017    2494.195     2.5e+04
x10         1.363e+04   6521.927      2.090      0.037     810.218    2.64e+04
x11         2.608e+04   8325.854      3.132      0.002    9710.218    4.24e+04
x12         1163.1257   6285.883      0.185      0.853   -1.12e+04    1.35e+04
x13         -766.9677   7391.603     -0.104      0.917   -1.53e+04    1.38e+04
x14        -3.096e+04   6434.499     -4.812      0.000   -4.36e+04   -1.83e+04
x15        -1209.2541   2128.847     -0.568      0.570   -5393.806    2975.297
x16        -3056.7131   3059.700     -0.999      0.318   -9070.988    2957.561
x17          858.2932   3783.165      0.227      0.821   -6578.055    8294.641
x18         5073.4433   7890.063      0.643      0.521   -1.04e+04    2.06e+04
x19          272.7980    289.807      0.941      0.347    -296.859     842.455
x20         7585.3392   4790.873      1.583      0.114   -1831.801     1.7e+04
x21        -3792.0023   4009.785     -0.946      0.345   -1.17e+04    4089.800
x22          206.5751    262.694      0.786      0.432    -309.787     722.937
x23         1128.1318    614.040      1.837      0.067     -78.850    2335.114
x24         3390.4238    883.851      3.836      0.000    1653.090    5127.758
x25         -356.3132    591.480     -0.602      0.547   -1518.951     806.325
x26          332.6481   4159.568      0.080      0.936   -7843.572    8508.868
x27        -2758.2165   4515.698     -0.611      0.542   -1.16e+04    6118.029
x28        -2.882e+04   1.16e+04     -2.479      0.014   -5.17e+04   -5972.950
x29        -4710.8143   5570.635     -0.846      0.398   -1.57e+04    6239.059
x30        -2853.0591   3869.168     -0.737      0.461   -1.05e+04    4752.340
x31         -661.0339   3988.241     -0.166      0.868   -8500.488    7178.420
x32        -6188.7444   5471.576     -1.131      0.259   -1.69e+04    4566.415
x33        -6970.7511   3668.512     -1.900      0.058   -1.42e+04     240.229
x34         2097.7886   4938.794      0.425      0.671   -7610.112    1.18e+04
x35         5256.3272   5066.167      1.038     